# Radar observations : rainfall composite product, rainfall

The aim of this notebook is to help you read the rainfall data.

For each radar data type, you will find one archive per year and per zone, each one sliced in periods of 10 or 11 days (each month is separated in 3 files). 

Each `.npz` file contains 3 arrays :

* `data` : an array containing maps of radar data values.

* `dates` : an array containing a datetime object and indicating the date of each `data` map.

* `miss_dates` : an array containing a datetime object for each missing radar map over the period.

Here, the `data` array contains a rainfall map for each time step. For each pixel, the value represents the cumulated rainfall in hundreth of millimeters (10<sup>-2</sup> mm). If the value is missing then the value is -1.

The time step between two radar scans is 5 min. The radar starts scanning at 00h every day and ends at 23h55. So each file of 11 days should contain a maximum of 3168 rainfall maps (minus the missing dates).

The radar's spatial resolution is 0.01° and the projection system used is EPSG:4326.

You will also find a `radar_coords_ZONE.npz` file containing 2 arrays `lats` and `lons`, indicating the coordinates (latitude and longitude) of each radar pixel.

# Quickstart

In [ ]:
from meteonet_toolbox.user_configuration import *
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import colors

First, let's select the zone and date we want to study. Use the `ind_min` and `ind_max` to select a smaller part of the data if you have a small RAM. The maximum value for `ind_max` is 3168 (1 record every 5 min during 11 days -> 12 records per hour * 24 hours * 11 days = 3168).

In [ ]:
zone = "NW"
year = 2016
month = 8
decade = 3      # Choose between 1,2,3 : part of the month we want to study
ind_min = 0
ind_max = 15

Get the file names :

In [ ]:
fname = "../../data_samples/radar/rainfall_%s_%s_%s.%s.npz" % (zone, str(year),"{:02d}".format(month),str(decade))
fname_coords = "../../data_samples/radar/radar_coords_%s.npz" % (zone)

Load the data :

In [ ]:
d = np.load(fname, allow_pickle=True)
data = d['data'][ind_min:ind_max,:,:]     #rainfall values -> subset of data from the first day day_i to the last day day_f
dates = d['dates'][ind_min:ind_max]        #associated dates values
miss_dates = d['miss_dates'][ind_min:ind_max]        #missing dates values

#get the coordinates of the points
coords = np.load(fname_coords, allow_pickle=True)
lat = coords['lats']
lon = coords['lons']

Plot the rainfall maps :

In [ ]:
title = "4 examples of rainfall plots"
fig, ax = plt.subplots(2, 2,figsize=(9,9))
fig.suptitle(title, fontsize=16)

if (np.max(data) > 65):
    borne_max = np.max(data)
else:
    borne_max = 65 + 10
cmap = colors.ListedColormap(['silver','white', 'darkslateblue', 'mediumblue','dodgerblue', 'skyblue','olive','mediumseagreen'
                              ,'cyan','lime','yellow','khaki','burlywood','orange','brown','pink','red','plum'])
bounds = [-1,0,2,4,6,8,10,15,20,25,30,35,40,45,50,55,60,65,borne_max]
norm = colors.BoundaryNorm(bounds, cmap.N)

pl=ax[0,0].pcolormesh(lon, lat, data[0,:,:],cmap=cmap, norm=norm)
ax[0,0].set_ylabel('latitude (degrees_north)')
ax[0,0].set_title(str(dates[0]) + " - "+  zone + " zone")

pl=ax[0,1].pcolormesh(lon, lat, data[1,:,:],cmap=cmap, norm=norm)
ax[0,1].set_title(str(dates[1]) + " - "+  zone + " zone")

pl=ax[1,0].pcolormesh(lon, lat, data[2,:,:],cmap=cmap, norm=norm)
ax[1,0].set_xlabel('longitude (degrees_east)')
ax[1,0].set_ylabel('latitude (degrees_north)')
ax[1,0].set_title(str(dates[2]) + " - "+  zone + " zone")

pl=ax[1,1].pcolormesh(lon, lat, data[3,:,:],cmap=cmap, norm=norm)
ax[1,1].set_xlabel('longitude (degrees_east)')
ax[1,1].set_title(str(dates[3]) + " - "+  zone + " zone")

cbar = fig.colorbar(pl,ax=ax.ravel().tolist(),cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, orientation= 'vertical').set_label('Rainfall (in 1/100 mm) / '+ str(-1) +' : missing values')
plt.show()

# Exploring the data

Dimensions of the 'data' array: 
<ul>
<li>time dimension (number of 5 min records) -> can be reduced and determined by min and max indexes when the dataset is loaded in memory (cf second cell)</li> 
<li>points number for latitude</li>
<li>points number for longitude</li>
</ul>    

In [ ]:
data.shape

Dimensions of the 'dates' array:
<ul>
    <li>time dimension (number of 5 min records for which there are data) -> can be reduced and determined by min and max indexes when the dataset is loaded in memory (cf second cell)</li>
</ul>

In [ ]:
dates.shape

Dimensions of the 'miss_dates' array:
<ul>
    <li>time dimension (dates for which there are no data) -> can be reduced and determined by min and max indexes when the dataset is loaded in memory (cf second cell)</li>
</ul>

In [ ]:
miss_dates.shape

Overview of 'data' values of type int16 :

In [ ]:
data

Overview of 'dates' of type datetime.datetime

In [ ]:
dates

Overview of 'miss_dates' of type datetime.datetime

In [ ]:
miss_dates

Overview of latitudes:

In [ ]:
lat

# Plots with Basemap

In [ ]:
from mpl_toolkits.basemap import Basemap, cm 

Define the boundaries of the area we want to plot :

In [ ]:
lllat=DOMAINS[zone]['lry']    #lower left latitude
urlat=DOMAINS[zone]['uly']    #upper right latitude
lllon=DOMAINS[zone]['ulx']    #lower left longitude
urlon=DOMAINS[zone]['lrx']    #upper right longitude

Select the index of the date we want to plot. /!\ 0 <= `ind` < `ind_max` - `ind_min` 

In [ ]:
ind = 5

Plot the rainfall values for 1 given time step

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(10,12))

#background map definition : coordinates of corners, resolution, projection type
m = Basemap(epsg=n_epsg,resolution='i', # You can change the resolution of the contours here
            llcrnrlat=lllat,
              urcrnrlat=urlat,
              llcrnrlon=lllon,
              urcrnrlon=urlon)

#colorbar definition
cmap = colors.ListedColormap(['silver','white', 'darkslateblue', 'mediumblue','dodgerblue', 'skyblue','olive','mediumseagreen'
                              ,'cyan','lime','yellow','khaki','burlywood','orange','brown','pink','red','plum'])
bounds = [-1,0,2,4,6,8,10,15,20,25,30,35,40,45,50,55,60,65,75]
norm = colors.BoundaryNorm(bounds, cmap.N)

#plot the data and the background map (coastlines and borders)
m.drawcoastlines()
m.drawcountries()
img=m.imshow(data[ind,:,:], interpolation='none', origin='upper',cmap=cmap, norm=norm)
plt.colorbar(img, cmap=cmap, norm=norm, boundaries=bounds, ticks=bounds, orientation= 'horizontal').set_label('Rainfall (in 1/100 mm) / '+ 
                                                                                                          str(-1) +' : missing values')
plt.title("Rainfalls - "+ str(dates[ind]) + " - "+  zone + " zone")
plt.show()